In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime

# Load Cifar10 dataset

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [33]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [7]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [8]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [9]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Build model

In [10]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [34]:
base_model = ResNet50(include_top=False, input_tensor=input_tensor)

In [36]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [37]:
resnet50 = Model(inputs=base_model.input, outputs=predictions)

In [38]:
resnet50.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [39]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [40]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [ ]:
resnet50.fit(x_train, y_train, validation_split=.2, batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 58s 1ms/step - loss: 1.7220 - acc: 0.4213 - val_loss: 1.2284 - val_acc: 0.5832
Epoch 2/10
40000/40000 [==============================] - 51s 1ms/step - loss: 0.9024 - acc: 0.6909 - val_loss: 1.0094 - val_acc: 0.6586
Epoch 3/10
40000/40000 [==============================] - 51s 1ms/step - loss: 0.5921 - acc: 0.7999 - val_loss: 0.9523 - val_acc: 0.6781
Epoch 4/10
40000/40000 [==============================] - 51s 1ms/step - loss: 0.3893 - acc: 0.8776 - val_loss: 0.9446 - val_acc: 0.6965
Epoch 5/10
40000/40000 [==============================] - 51s 1ms/step - loss: 0.2553 - acc: 0.9276 - val_loss: 0.9539 - val_acc: 0.7021
Epoch 6/10
40000/40000 [==============================] - 50s 1ms/step - loss: 0.1619 - acc: 0.9596 - val_loss: 0.9822 - val_acc: 0.7044
Epoch 7/10
40000/40000 [==============================] - 50s 1ms/step - loss: 0.1059 - acc: 0.9783 - val_loss: 1.0256 - val_acc

In [24]:
resnet50.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 11s 1ms/step


[1.0069275924682617, 0.7673]